#   Maquinas de vectores de soporte

In [10]:
"""
Construiremos un clasificador de máquinas de vectores de soporte 
para predecir el nivel de ingresos de una persona determinada
en función de 14 atributos. Nuestro objetivo es ver 
dónde el ingreso es mayor o menor que $ 50,000 por año
 https://archive.ics.uci.edu/ml/datasets/Census+Income.
"""

'\nConstruiremos un clasificador de máquinas de vectores de soporte \npara predecir el nivel de ingresos de una persona determinada\nen función de 14 atributos. Nuestro objetivo es ver \ndónde el ingreso es mayor o menor que $ 50,000 por año\n https://archive.ics.uci.edu/ml/datasets/Census+Income.\n'

In [86]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [112]:
#datos
input_file = 'income_data.txt'
X = []
y =  []
count_class1 = 0
count_class2 = 0
max_datapoints = 25000
data = np.load

In [113]:
#Abrir el archivo y leer por lineas
with open(input_file, 'r') as f:
    for  line in f.readlines():
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
            break
        
        if '?' in line:
            continue
        data = line[:-1].split(', ')
        
        if data[-1] == '<=50K' and count_class1 < max_datapoints:
            X.append(data)
            count_class1 += 1
        if data[-1] == '>50K' and count_class2 < max_datapoints:
            X.append(data)
            count_class2 += 1
            
#Convierte  a un numpy  array para poder ser procesado por sklearn
X = np.array(X)  

In [114]:
#Codifica los campos String en numeros
label_encoder = []
X_encoded = np.empty(X.shape)
for i, item in enumerate(X[0]):
    if item.isdigit():
        X_encoded[:,i] = X[:, i]
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])
X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int) 


In [115]:
#Crea el SVM con  linear kernel
classifier = OneVsOneClassifier(LinearSVC(random_state=0),n_jobs=-1)
#Entredando clasificador
classifier.fit(X,y)

OneVsOneClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=-1)

In [116]:
#Validación cruzada 
X_train, X_test, y_train, y_test = train_test_split(X,y,
test_size=0.2, random_state=5)
classifier = OneVsOneClassifier(LinearSVC(random_state=0),n_jobs=-1)
classifier.fit(X_train, y_train)
y_test_pred = classifier.predict(X_test)

In [117]:
#Puntuar F1 de clasificador 
f1 = cross_val_score(classifier,X,y,
scoring='f1_weighted',cv=3)
print("F1 score: " + str(round(100*f1.mean(),2)) + "%") 

F1 score: 71.35%


In [125]:
input_data = ['37', 'Private', '215646', 'HS-grad', '9', 'Never-married', 'Handlers-cleaners', 'Not-in-family', 'White', 'Male', '0', '0', '40', 'United-States']
"""
Uso el ultimo de los datos reales disponibles
que no se uso para el entrenamient de la IA
para probar la predicción que segun la evaluación es 
71.35% acertada
"""
input_data_real = ["52", "Self-emp-inc", "287927", "HS-grad", "9", "Married-civ-spouse", "Exec-managerial", "Wife", "White", "Female", "15024", "0", "40", "United-States"]
# Encode test datapoint
input_data_encoded = [[-1] * len(input_data),[-1] * len(input_data)]
count = 0
for i, item in enumerate(input_data):
    if item.isdigit():
        input_data_encoded[0][i] = int(input_data[i])
        input_data_encoded[1][i] = int(input_data_real[i])
    else:
        #print(label_encoder[count])
        #print([input_data[i]])
        #print(label_encoder[count].transform([np.array(input_data[i])]))
        input_data_encoded[0][i] = int(label_encoder[count].transform([input_data[i]]))
        input_data_encoded[1][i] = int(label_encoder[count].transform([input_data_real[i]]))
        count += 1
input_data_encoded = np.array(input_data_encoded)
#prediccion con dato aleatorio sera la posición 0, y la real  1
predicted_class = classifier.predict(input_data_encoded)
print(label_encoder[-1].inverse_transform(predicted_class))

['<=50K' '>50K']
